# Imports and Load Inital Datasets

In [62]:
# Create a data directory in our project folder
import os
os.makedirs('Data/', exist_ok=True)

In [63]:
os.listdir('Data/')

['IMDB Movie Dataset Info.docx',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz']

In [64]:
import pandas as pd
import numpy as np
us = pd.read_csv('Data/title-akas-us-only.csv', low_memory=False)
basics = pd.read_csv('Data/title.basics.tsv.gz', sep='\t', low_memory=False)
ratings = pd.read_csv('Data/title.ratings.tsv.gz', sep='\t', low_memory=False)

In [65]:
us.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [66]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10017011 entries, 0 to 10017010
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 687.8+ MB


In [67]:
us['region'].value_counts()

US    1452564
Name: region, dtype: int64

# Filtering Basics Data

In [68]:
us_basics_filter = basics['tconst'].isin(us['titleId'])
basics = basics[us_basics_filter]
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1365643 entries, 0 to 10016966
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   tconst          1365643 non-null  object
 1   titleType       1365643 non-null  object
 2   primaryTitle    1365643 non-null  object
 3   originalTitle   1365643 non-null  object
 4   isAdult         1365643 non-null  object
 5   startYear       1365643 non-null  object
 6   endYear         1365643 non-null  object
 7   runtimeMinutes  1365643 non-null  object
 8   genres          1365643 non-null  object
dtypes: object(9)
memory usage: 104.2+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"


In [69]:
# The data dict mentions that null values were replaced with '\N', we will correct this now
basics = basics.replace({'\\N':np.nan})
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            0
originalTitle           0
isAdult                 0
startYear           98665
endYear           1328513
runtimeMinutes     503119
genres              28616
dtype: int64

## Dropping Null Values

In [70]:
# Dropping rows with null values in the runtimeMinutes and genres columns
basics = basics.dropna(subset=['runtimeMinutes', 'genres'])
basics.isna().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear           8802
endYear           833124
runtimeMinutes         0
genres                 0
dtype: int64

## Filtering for Movies Only

In [71]:
movie_type = basics['titleType'] == 'movie'
movie_type.sum()

203476

In [82]:
basics = basics[movie_type]

## Filtering Year

In [83]:
basics['startYear'] = basics['startYear'].astype(float)

In [84]:
basics['startYear'].dtype

dtype('float64')

In [85]:
basics['endYear'] = basics['endYear'].astype(float)
basics['endYear'].dtype

dtype('float64')

In [86]:
years_filter = (basics['startYear'] >= 2000) & (basics['startYear'] <=2022)
years_filter.value_counts()

True     121127
False     82349
Name: startYear, dtype: int64

## Filtering Documentary

In [93]:
filter_docu = basics['genres'].str.contains('Documentary')
basics = basics[~filter_docu]

In [94]:
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86979 entries, 34802 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86979 non-null  object 
 3   originalTitle   86979 non-null  object 
 4   isAdult         86979 non-null  object 
 5   startYear       86979 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  86979 non-null  object 
 8   genres          86979 non-null  object 
dtypes: float64(2), object(7)
memory usage: 6.6+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [95]:
basics_csv = basics.to_csv('Data/basics_filtered.csv', index=True)

# Filtering Ratings

In [96]:
ratings.info()
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1331492 entries, 0 to 1331491
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1331492 non-null  object 
 1   averageRating  1331492 non-null  float64
 2   numVotes       1331492 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.5+ MB


,tconst,averageRating,numVotes
0,tt0000001,5.7,1988
1,tt0000002,5.8,265
2,tt0000003,6.5,1849
3,tt0000004,5.5,178
4,tt0000005,6.2,2632


In [99]:
filter_ratings = ratings['tconst'].isin(basics['tconst'])
ratings = ratings[filter_ratings]
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71900 entries, 17961 to 1331462
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         71900 non-null  object 
 1   averageRating  71900 non-null  float64
 2   numVotes       71900 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [100]:
ratings = ratings.replace({'\\N':np.nan})
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [101]:
ratings.info()
ratings.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71900 entries, 17961 to 1331462
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         71900 non-null  object 
 1   averageRating  71900 non-null  float64
 2   numVotes       71900 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


,tconst,averageRating,numVotes
17961,tt0035423,6.4,87153
40764,tt0062336,6.4,175
46645,tt0069049,6.7,7754
63640,tt0088751,5.2,336
69953,tt0096056,5.6,846


In [102]:
ratings_csv = ratings.to_csv('Data/ratings_filtered.csv')